<a href="https://colab.research.google.com/github/ANNE513/first-group/blob/main/11_16%E6%9C%80%E7%B5%82%E6%A8%A1%E5%9E%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ============================================================
# 📦 Step 0: transformers 版本檢查
# ============================================================
import importlib, subprocess, sys

def get_training_args():
    import transformers
    version = transformers.__version__
    print(f"🔍 transformers 版本: {version}")
    major = int(version.split('.')[0])

    from transformers import TrainingArguments

    if major >= 4:
        return TrainingArguments(
            output_dir="./results",
            eval_strategy="epoch",
            save_strategy="epoch",
            learning_rate=3e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=5,
            weight_decay=0.01,
            logging_dir="./logs",
            logging_steps=50,
            report_to="none",
        )
    else:
        return TrainingArguments(
            output_dir="./results",
            do_eval=True,
            save_strategy="epoch",
            learning_rate=3e-5,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=2,
            weight_decay=0.01,
            logging_dir="./logs",
            logging_steps=50,
            report_to="none",
        )

try:
    import transformers
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "transformers", "accelerate", "datasets"])
    import transformers

# ============================================================
# Step 1: 上傳訓練資料
# ============================================================
from google.colab import files
import pandas as pd

print("📤 請上傳 train_split.xlsx 與 test_split.xlsx...")
uploaded = files.upload()

if "train_split.xlsx" not in uploaded or "test_split.xlsx" not in uploaded:
    raise ValueError("❌ 請必須上傳 train_split.xlsx + test_split.xlsx 才能訓練！")

train_df = pd.read_excel("train_split.xlsx")
test_df = pd.read_excel("test_split.xlsx")

print(f"📊 訓練集筆數: {len(train_df)}, 測試集筆數: {len(test_df)}")

# ============================================================
# Step 2: Tokenizer / Dataset
# ============================================================
import torch, random, numpy as np
from collections import Counter
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from torch.nn import CrossEntropyLoss
from sklearn.metrics import accuracy_score, f1_score, classification_report

device = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
set_seed()

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")

def encode_texts(texts):
    return tokenizer(texts, truncation=True, padding=True, max_length=256)

train_enc = encode_texts(train_df["content_cleaned"].tolist())
test_enc  = encode_texts(test_df["content_cleaned"].tolist())

class FoodDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = FoodDataset(train_enc, train_df["label"].tolist())
test_dataset  = FoodDataset(test_enc,  test_df["label"].tolist())

# ============================================================
# Step 3: Class weights + 模型
# ============================================================
label_counts = Counter(train_df["label"].tolist())
total = sum(label_counts.values())
weights = [total / label_counts[i] for i in range(2)]
class_weights = torch.tensor(weights, dtype=torch.float).to(device)
print(f"🔸 Class Weights: {class_weights}")

model = AutoModelForSequenceClassification.from_pretrained(
    "hfl/chinese-roberta-wwm-ext",
    num_labels=2
)

def weighted_loss(outputs, labels):
    logits = outputs.logits
    loss_fct = CrossEntropyLoss(weight=class_weights)
    return loss_fct(logits, labels)

training_args = get_training_args()

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        loss = weighted_loss(outputs, labels)
        return (loss, outputs) if return_outputs else loss

def compute_metrics(pred):
    preds = np.argmax(pred.predictions, axis=-1)
    return {
        "accuracy": accuracy_score(pred.label_ids, preds),
        "f1": f1_score(pred.label_ids, preds),
    }

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# ============================================================
# Step 4: 開始訓練
# ============================================================
print("\n🚀 開始訓練模型...")
trainer.train()

model.save_pretrained("./food_model")
tokenizer.save_pretrained("./food_model")
print("💾 模型已成功儲存到 /food_model")

# ============================================================
# Step 5: 訓練完成 → 自動進入預測模式（含信心分數）
# ============================================================
print("\n🔮 訓練完成！請上傳 predict_sample.csv 進行預測...")
from google.colab import files
uploaded2 = files.upload()

if "predict_sample.csv" not in uploaded2:
    raise ValueError("❌ 請上傳 predict_sample.csv 用於預測！")

import pandas as pd
import torch
import numpy as np

# 讀取預測資料
pred_df = pd.read_csv("predict_sample.csv")
texts = pred_df["content_cleaned"].astype(str).tolist()

# 編碼
enc = tokenizer(
    texts,
    truncation=True,
    padding=True,
    max_length=256,
    return_tensors="pt"
)
enc = {k: v.to(device) for k, v in enc.items()}
model.to(device)
model.eval()

# ========= 🔢 預測 + 信心分數 =========
with torch.no_grad():
    outputs = model(**enc)
    logits = outputs.logits                    # [num_samples, num_labels]
    probs = torch.softmax(logits, dim=1)       # 轉成機率
    probs_np = probs.cpu().numpy()
    preds = np.argmax(probs_np, axis=1)        # 取最大機率的類別

# ========= 🧾 把結果寫回 DataFrame =========
pred_df["prediction"] = preds
pred_df["prob_0"] = probs_np[:, 0]             # 類別 0 的機率
pred_df["prob_1"] = probs_np[:, 1]             # 類別 1 的機率

# 信心分數：取兩個類別機率的最大值
pred_df["confidence"] = pred_df[["prob_0", "prob_1"]].max(axis=1)

# 也可以看「模型有多不確定」= 1 - confidence（數字越大越不確定）
pred_df["uncertainty"] = 1 - pred_df["confidence"]

# 中文標籤（可依你原本定義調整）
pred_df["prediction_label"] = pred_df["prediction"].map({
    0: "✅ 符合法規",
    1: "⚠️ 不符合法規"
})

print("\n📊 部分預測結果（含信心分數）：")
print(pred_df[["content_cleaned", "prediction", "prediction_label", "prob_0", "prob_1", "confidence"]].head())

# ========= 🔻 依「信心由低到高」排序（最容易錯的在最上面） =========
pred_sorted = pred_df.sort_values(by="confidence", ascending=True)

# ========= 💾 存檔 & 下載 =========
pred_df.to_csv("predict_output_with_confidence.csv", index=False)
pred_sorted.to_csv("predict_output_sorted_by_confidence.csv", index=False)

files.download("predict_output_with_confidence.csv")
files.download("predict_output_sorted_by_confidence.csv")

print("\n💾 已下載：")
print(" - predict_output_with_confidence.csv（原順序 + 信心分數）")
print(" - predict_output_sorted_by_confidence.csv（信心由低到高排序）")


📤 請上傳 train_split.xlsx 與 test_split.xlsx...


Saving test_split.xlsx to test_split.xlsx
Saving train_split.xlsx to train_split.xlsx
📊 訓練集筆數: 1233, 測試集筆數: 309


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


💾 已下載：
 - predict_output_with_confidence.csv（原順序 + 信心分數）
 - predict_output_sorted_by_confidence.csv（信心由低到高排序）
